In [1]:
# Combine Data Across All Sessions (for a given participant)

In [2]:
import pandas as pd
import numpy as np
from pandas import *
from scipy import stats
import os
import re

In [3]:
# get column names & types from sample file

In [4]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Data/All Synchronized/P3_annos/annos_s20_2017-11-08/other_data/Combined_s20_2017-11-08.csv'

init_data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
of_op_cols = []

for i in init_data.columns:
    if 'of' in i or 'op' in i:
        of_op_cols.append(i)

save_columns = ['engagement', 'timestamp', 'participant', 'session_num', 'session_date', 'activity', 'difficulty', 'ros_ROBOT_STATE', 'ros_PARTICIPANT_STATE', 'transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2']

new_columns = ['mistake_made', 'game_incorrect', 'game_correct', 'game_start']

master_cols = save_columns + new_columns + of_op_cols

In [6]:
master = pd.DataFrame(columns = master_cols)

### Precondition Calculations
- game start
    - new continuous instance of "game2_act1_inst0" in ros_ROBOT_STATE
- game end (moving on)
    - "Movingon" in ros_PARTICIPANT_STATE
- game end (completed successfully)
    - "GameCompleted" in ros_PARTICIPANT_STATE
- mistake made
    - new continuous instance of "Madeamistake in ros_PARTICIPANT_STATE"

In [7]:
curr_sessions = [
's2-1',
's2-2',
's2-3',
's3-1',
's3-2',
's4-2',
's4-3',
's4-4',
's5-1',
's5-2',
's7-1',
's7-2',
's8',
's9'
]

In [8]:
rootdir = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Data/All Synchronized/P10_annos'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if 's2-3_2018-07-08.csv' in file:
            continue
        
        found = False
        for i in curr_sessions:
            if i in file:
                found = True
        
        if found:
            continue
            
        if 'Combined' in file: #one of the files to process
            print(file)
                        
            path_to_file = subdir + '/' + file
            data = pd.read_csv(path_to_file)
            
            new_columns = ['mistake_made', 'game_incorrect', 'game_correct', 'game_start', 'session_date', 'session_num']

            for c in new_columns:
                if c != 'session_date':
                    data[c] = 0.0
                else:
                    data[c] = ''
            
            for i,r in data.iterrows():
                ros_robot = r['ros_ROBOT_STATE']
                ros_participant = r['ros_PARTICIPANT_STATE']

                # new continuous instance in ros_ROBOT_STATE 
                if (isinstance(ros_robot, str)) and ((i==0) or (data.at[i-1, 'ros_ROBOT_STATE'] != ros_robot)): 
                    # new game_start
                    if ('game' in ros_robot) and ('act' in ros_robot) and  ('inst' in ros_robot):
                        data.at[i, 'game_start'] = 1

                # new continuous instance in ros_PARTICIPANT_STATE
                if (isinstance(ros_participant, str)) and ((i==0) or (data.at[i-1, 'ros_PARTICIPANT_STATE'] != ros_participant)):
                    # new game correct 
                    if ('GameCompleted' in ros_participant):
                        data.at[i, 'game_correct'] = 1

                    # new game incorrect
                    if ('Movingon' in ros_participant):
                        data.at[i, 'game_incorrect'] = 1

                    # mistake made
                    if ('Madeamistake' in ros_participant):
                        data.at[i, 'mistake_made'] = 1
                        
                # extract session num and date
                session = r['session']
                nan_indices = []
                
                if (isinstance(session, str)):
                    date_regex = '201\d-\d\d-\d\d'
                    date = re.findall(date_regex, session)[0]

                    sess_regex1 = 's\d*_'
                    sess_regex2 = 's\d*-\d*_'

                    res1 = re.findall(sess_regex1, session)
                    res2 = re.findall(sess_regex2, session)

                    if (res1):
                        num = res1[0]
                        num = float(num[1:-1])
                    else:
                        num = res2[0]
                        num = num[1:-1]
                        num = num.replace('-', '.')
                        num = float(num)

                    data.at[i, 'session_num'] = num
                    data.at[i, 'session_date'] = date
                else:
                    nan_indices.append(i)
            
            data = data.drop(data.index[nan_indices])
            data = data.drop(columns=['session'])
            
            data['timestamp'] = data['Timestamp']
            data = data.drop(columns=['Timestamp'])
            
            print(data.shape)
            print(master.shape)
            
            try:
                master = pd.concat([master,data], sort=True)
            except:
                print('error with this file', file)
                continue

Combined_s17-2_2018-07-28.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(22602, 1553)
(0, 1553)
Combined_s12-1_2018-07-22.csv
(23870, 1553)
(22602, 1553)
Combined_s22-2_2018-08-06.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(12008, 1553)
(46472, 1553)
Combined_s21_2018-08-02.csv


...

(24931, 1553)
(227026, 1553)
Combined_s15-1_2018-07-25.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(25719, 1553)
(251957, 1553)
Combined_s14-2_2018-07-24.csv
(23068, 1553)
(277676, 1553)
Combined_s13_2018-07-23.csv
(20827, 1553)
(300744, 1553)
Combined_s12-2_2018-07-22.csv
(47513, 1553)
(321571, 1553)
Combined_s11_2018-07-21.csv
(48088, 1553)
(369084, 1553)
Combined_s10-3_2018-07-18.csv


/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(45192, 1550)
(417172, 1553)
Combined_s10-1_2018-07-18.csv
(14917, 1553)
(462364, 1553)


In [9]:
data.columns

Index(['participant', 'activity', 'difficulty', 'ros_ROBOT_STATE',
       'ros_PARTICIPANT_STATE', 'transcript_spk_1', 'transcript_spk_2',
       'transcript_spk_0', 'engagement', 'of_frame',
       ...
       'op_p2_hand_right_x20', 'op_p2_hand_right_y20', 'op_p2_hand_right_c20',
       'mistake_made', 'game_incorrect', 'game_correct', 'game_start',
       'session_date', 'session_num', 'timestamp'],
      dtype='object', length=1553)

In [10]:
master.shape

(477281, 1553)

In [11]:
master.columns

Index(['activity', 'difficulty', 'engagement', 'game_correct',
       'game_incorrect', 'game_start', 'mistake_made', 'of_AU01_c',
       'of_AU01_r', 'of_AU02_c',
       ...
       'op_p2_pose_y9', 'participant', 'ros_PARTICIPANT_STATE',
       'ros_ROBOT_STATE', 'session_date', 'session_num', 'timestamp',
       'transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'],
      dtype='object', length=1553)

In [12]:
path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p10_data1-2.csv'

master.to_csv(path_data, index=False)